# Sensitivity Sampling and Analysis

Sensitivity is a measure of the proportion of variance in some defined value that is explained by a particular input parameter.

### This notebook demonstrates the use of the Sobol sampling and associated sensitivity analysis of the model

see https://en.wikipedia.org/wiki/Variance-based_sensitivity_analysis  and https://en.wikipedia.org/wiki/Sobol_sequence for some background
    
####     - It specifies the Sobol sampling (SobolOptimizer) as part of the Optimizer specifications (OptimizerSpec)
####     - This is is place of the standard SCE optimisation used typically in calibration


#### It assumes the opimisation/sampling is beign undertaken on a cluster (e.g. NCI) and runs catchment/cell simulations in parallel on differing compute nodes

## Sensitivity analysis background

Sensitivity cannot be measured directly, but rather must be estimated from a sample distribution.
This estimation requires selection of a sampling scheme (Sobol sequence here), and corresponding variance estimation functions.
See the following reference for further background: 


   - ***Andrea Saltelli, Paola Annoni, Ivano Azzini, Francesca Campolongo, Marco Ratto, Stefano Tarantola. Variance based sensitivity analysis of model output. Design and estimator for the total sensitivity index, In Computer Physics Communications, Volume 181, Issue 2, 2010, Pages 259-270, ISSN 0010-4655, https://doi.org/10.1016/j.cpc.2009.09.018.***

 

This notebook goes through the following steps:

1. Import required libraries
2. Set up calibration configuration <br>

   - 2.1 Catchments to be calibrated
   - 2.2 Period to calibrate over<br>
   - 2.3 Import model/get default mapping<br>
   - 2.4 Setting the optimizer - SobolOptimizer<br>
   - 2.5 Define objective functions<br>
   - 2.6 Update forcing paths (optional)<br>
   - 2.7 Combine into a specification dictionary<br>
   
   
3. Run 'calibration'; ie sampling<br>
4. Visualise sensitivity outputs

### 1. Import required libraries

In [ ]:
from os.path import join

import h5py

from awrams.calibration import support
from awrams.utils import config_manager, extents, gis
from awrams.utils import datetools as dt

### 2. Define calibration configuration

#### 2.1 Select catchment extents

In [ ]:
# Point to some catchment data
system_profile = config_manager.get_system_profile()
system_settings = system_profile.get_settings()

base_data_path = system_settings['DATA_PATHS']['BASE_DATA']
catchment_shapefile = join(base_data_path, 'spatial/shapefiles/Final_list_all_attributes.shp')
calvalshapefile = gis.ShapefileDB(catchment_shapefile)

# Define the extenst of the calibration
def_extent = extents.get_default_extent() 

In [ ]:
 ## Create a dict with multiple extents
extent_map = {}
cal_catchments = ['109001','111101','112102']

for catchment in cal_catchments:
    extent_map[catchment] = calvalshapefile.get_extent_by_field('StationID', catchment, parent_extent=def_extent)
    
extent_map

#### 2.2 Specify running period

In [ ]:
run_period = dt.dates('2009 - 2011')
eval_period = dt.dates('2009 - 2011')

#### 2.3 Import model/get default mapping

In [ ]:
# Import the model and get input map
model_profile = config_manager.get_model_profile('awral', 'v6_default')

model_settings = model_profile.get_settings()
input_map = model_profile.get_input_mapping(model_settings)
model = model_profile.get_model()

#### 2.4 Specify the optimizer

In [ ]:
# Use the SobolOptimizer class to generate Sobol sequence samples

from awrams.calibration.sensitivity import SobolOptimizer

# Set termination conditions - 3000 max evaluations is not really a 'proper' run, but will be enough to see some results...
# Threshold is the convergence threshold for the sum of estimated sensitivities; ie the run should terminate once
# these values stabilise
optimizer_spec = support.OptimizerSpec(SobolOptimizer, threshold=0.01, max_eval=3000)

#### 2.5 Set up objective functions<br>
When used with a sampling optimizer (eg Sobol), objective functions are not directly opimized on, but are still needed in order to obtain logging of the result

### see prespecified objective function  for this example [test_objectives.py]

 class ***LocalQTotal*** provides Simple sum of Qtot, Etot and Dd from awrams.calibration.objectives import test_objectives
 
 ### Testing below using this objective determines sensitivity of these total runoff, ET and deep draiange to different parameters
      
     
        
     

[test_objectives.py]:../../../edit/calibration/awrams/calibration/objectives/test_objectives.py



In [ ]:
from awrams.calibration.objectives import test_objectives as tobj

# Our 'objective functions' (ie what gets stored in the logfile)
# are volume totals of water balance outputs; runoff, et, and deep drainage

local_objfspec = support.ObjectiveFunctionSpec(tobj.LocalQTotal)
global_objfspec = tobj.GlobalQTotal

observations = {}
objective_spec = support.ObjectiveSpec(global_objfspec, local_objfspec, observations, eval_period)

#### 2.7 Build spec dictionary

Assemble above settings into specification dictionary

In [ ]:
'''
User specifiable calibration description
'''
cal_spec = {}
cal_spec['optimizer_spec'] = optimizer_spec
cal_spec['objective_spec'] = objective_spec
cal_spec['extent_map'] = extent_map
cal_spec['run_period'] = run_period
cal_spec['model'] = model
cal_spec['node_mapping'] = input_map
cal_spec['logfile'] = './sobolres4.h5'

### 3. Run the calibration

In [ ]:
from awrams.calibration import cluster
# write calibration specificatiom to a pickle file
nnodes = 1
ncores = 4
_ = cluster.build_pickle_from_spec(cal_spec, ncores, nnodes, 'test_sens.pkl')

In [ ]:
# read calibration specification pickle file and run solbol sampler using run_from_pickle()
from awrams.calibration.launch_calibration import run_from_pickle
run_from_pickle('./test_sens.pkl')

### 4. Examine the results

Results are stored in an HDF5 file (like all AWRAMCMS calibration outputs).  SensitivityResults provides a wrapper to directly perform sensitivity analysis on the results of a SobolOptmizer run

In [ ]:
from matplotlib import pyplot as plt

from awrams.calibration import sensitivity
from awrams.calibration.sensitivity import SensitivityResults

In [ ]:
#Open the results

sr = SensitivityResults('./sobolres4.h5')

In [ ]:
#Show the global outputs that are available

sr.global_keys

In [ ]:
# show the catchments
sr.catchment_ids

### View overall sensitivity of different outputs (dd_vol, etot_vol, qtot_vol) to individual parameters 

In [ ]:
%matplotlib inline
#Get the sensitivities for all catchments (get_all_catchment_si), and plot their distribution
catchsens = sr.get_all_catchment_si('qtot_vol')
plt.figure(figsize=(12, 3))
plt.title('Sensitivity index for all catchments: Total runoff (Qtot)')
bp = catchsens.astype(float).boxplot(return_type='axes', rot=45)
plt.ylabel('Sensitivity index [-]')
plt.tight_layout()

catchsens = sr.get_all_catchment_si('etot_vol')
plt.figure(figsize=(12, 3))
plt.title('Sensitivity index for all catchments: Total actual evapotranspiration (Etot)')
bp = catchsens.astype(float).boxplot(return_type='axes', rot=45)
plt.ylabel('Sensitivity index [-]')
plt.tight_layout()

catchsens = sr.get_all_catchment_si('dd_vol')
plt.figure(figsize=(12, 3))
plt.title('Sensitivity index for all catchments: Deep drainage (Dd)')
bp = catchsens.astype(float).boxplot(return_type='axes', rot=45)
plt.ylabel('Sensitivity index [-]')
plt.tight_layout()

In [ ]:
# Convenience functions for plotting the sensitivity indices (si and sti) 
# sti deals with the interactions between parameters, whereas si does not
#
# Inputs: 
#       sr= sensitivity samppling results
#       param=  variable for analysis (from sr.global_keys)
#       catchment=  catchment is not specified, provides the global value, else for the specified catchment

def plot_si(sr, param, catchment=None):
    fig = plt.figure(figsize=(10, 2.5))
    if catchment is not None:
        psens = sr.get_catchment_si(param, catchment)
        title = '%s (Si), catch_id: %s' % (param, catchment)
    else:
        psens = sr.get_global_si(param)
        title = '%s (Si), global' % param
    psens.plot(kind='bar', xlim=[0, psens.max() + 0.1], title=title, rot=45)
    #plt.gca().invert_yaxis()
    
    print(sum(psens))
    

def plot_sti(sr, param, catchment=None):
    fig = plt.figure(figsize=(10, 2.5))
    if catchment is not None:
        psens = sr.get_catchment_si(param, catchment, True)
        title = '%s (STi), catch_id: %s' % (param, catchment)
    else:
        psens = sr.get_global_si(param, True)
        title = '%s (STi), global' % param
    psens.plot(kind='bar', xlim=[0, psens.max() + 0.1], title=title, rot=45)
    #plt.gca().invert_yaxis()
    print(sum(psens))    

In [ ]:
# Show the global sensitivity index for qtot
plot_sti(sr, 'qtot_vol')
plot_si(sr, 'qtot_vol')
#Get the sensitivities for individual catchments, and plot their distribution
# Show the total sensitivity index for runoff
plot_sti(sr, 'qtot_vol', '112102')
#Get the sensitivities for individual catchments, and plot their distribution
# Show the total sensitivity index for runoff
plot_sti(sr, 'qtot_vol', '109001')

In [ ]:
# Show the total sensitivity index for the three outputs
plot_sti(sr, 'qtot_vol')
plot_sti(sr, 'etot_vol')
plot_sti(sr, 'dd_vol')